In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", RuntimeWarning)

import sys
sys.path.append('../../dstorch/')

import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
from dstorch import transforms
from dstorch.train import train
import torch.backends.cudnn as cudnn
import cv2

In [ ]:
ids = list([x.split('.')[0] for x in list(next(os.walk("../output_data/without_scale_05/images/"))[2])])
classes = pd.read_csv("../data/classes.csv")
val_ids = list(classes.loc[classes['type'] == 'val', 'id'])
train_ids = list(set(ids) - set(val_ids))

In [ ]:
train_transforms = transforms.DualCompose(
    [
        transforms.RandomCrop((256, 256)),
        transforms.VerticalFlip(),
        transforms.HorizontalFlip(),
        transforms.Transpose(),
        transforms.ShiftScaleRotate(scale_limit=0.6, shift_limit=0.2),
        transforms.Distort1(0.5, 0.5),
        transforms.ImageOnly(transforms.RandomBrightness(0.5)),
        transforms.ImageOnly(transforms.RandomContrast(0.5)),
        transforms.ImageOnly(transforms.RandomSaturation(0.5)),
        transforms.ImageOnly(transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ))
    ]
)

val_transforms = transforms.DualCompose(
    [
        transforms.ImageOnly(transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ))
    ]
)

In [ ]:
cudnn.benchmark = True

In [ ]:
train_args = {
    'model_name': 'TernausNet34_without_scale_05_epoch_500',
    'network': 'TernausNet34',
    'nb_epoch': 500,
    'loss': 'BCEDiceLossMulti',
    'lr_args': {
        'adjust_init_lr': 0.0003,
        'adjust_lr_decay_factor': 0.5,
        'cycle_epochs_decay': 2,
        'cycle_init_lr': 0.001,
        'cycle_lr_decay_factor': 0.3,
        'global_cycle_lr_decay_factor': 0.7,
        'global_num_epochs_per_cycle': 100,
        'num_epochs_per_cycle': 5,
        'num_epochs_per_decay': 100
    }, 
    'model_dir': 'models', 
    'log_dir': 'logs', 
    'network_args': {
        'num_classes': 3, 
        'num_filters': 16, 
        'pretrained': True
    }, 
    'loss_args': {
        'weights': [0.5, 0.4, 0.1],
        'num_classes': 3
    },       
    'optimizer': 'Adam'
}

train(
    train_args, train_ids + val_ids, val_ids[:1], 
    "../output_data/without_scale_05/images/{}.png", "../output_data/without_scale_05/masks/{}.png", 
    16, 1, train_transforms, val_transforms, period=64
)